## Histogram Equalization

In [1]:
import cv2 
import numpy as np 
inp = input("Enter the path of the image: ") 
img = cv2.imread(inp, 0) 

equ = cv2.equalizeHist(img) 
res = np.hstack((img, equ)) 

cv2.imshow(res)
# save the image
cv2.imwrite('res.png', res)
  
cv2.waitKey(0) 
cv2.destroyAllWindows() 

## Ground Control Points (GCPs)

In [ ]:

import rasterio
from rasterio.transform import from_gcps

# ... define your GCPs as above
gcps = [
    (100, 200, 45.123, -75.456, 100.0),  # (pixel_x, pixel_y, lon, lat, elevation)
    (500, 300, 45.124, -75.457, 110.0),
    # ... more GCPs
]

# Create an affine transformation matrix from GCPs
transform = from_gcps(gcps)

# Open the image you want to georeference
with rasterio.open("image.tif") as src:
    # Update the image's metadata with the new transformation
    profile = src.profile
    profile.update(transform=transform, crs="EPSG:4326")  # Set desired CRS

    # Write the georeferenced image to a new file
    with rasterio.open("georeferenced_image.tif", "w", **profile) as dst:
        dst.write(src.read())


## Denoising

In [ ]:
# Importing the required libraries and the dataset
import numpy 
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
# Formatting the dataset for keras
num_pixels = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape(X_train.shape[0], num_pixels).astype('float32')
X_test = X_test.reshape(X_test.shape[0], num_pixels).astype('float32')
X_train = X_train / 255
X_test = X_test / 255

# Adding noise to the dataset
noise_factor = 0.2
x_train_noisy = X_train + noise_factor * numpy.random.normal(loc=0.0, scale=1.0, size=X_train.shape)
x_test_noisy = X_test + noise_factor * numpy.random.normal(loc=0.0, scale=1.0, size=X_test.shape)
x_train_noisy = numpy.clip(x_train_noisy, 0., 1.)
x_test_noisy = numpy.clip(x_test_noisy, 0., 1.)

In [ ]:
# create model
model = Sequential()
model.add(Dense(500, input_dim=num_pixels, activation='relu'))
model.add(Dense(300, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(300, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(784, activation='sigmoid'))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Training model
model.fit(x_train_noisy, X_train, validation_data=(x_test_noisy, X_test), epochs=2, batch_size=200)